In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql import functions
from pyspark.sql.functions import *

In [3]:
# Criando uma SparkSession. AppName - Identifica o nome do meu app. Master - qtd de partições que o spark irá criar.
spark = SparkSession.builder \
    .appName('Case - Parte 2') \
    .master('local[3]') \
    .getOrCreate()

In [4]:
#Lendo todos os arquivos json para um único dataFrame, pois todos possuem a mesma estrutura sobre PageViews
dfPageViews = spark.read.json(['Datasets - Teste Data Engineer - Passei Direto/BASE B/*.json'])

In [5]:
#Verficar todas as colunas que o dataFrame Possui e seus tipos de dados
dfPageViews.printSchema()

root
 |-- Last Accessed Url: string (nullable = true)
 |-- Page Category: string (nullable = true)
 |-- Page Category 1: string (nullable = true)
 |-- Page Category 2: string (nullable = true)
 |-- Page Category 3: string (nullable = true)
 |-- Page Name: string (nullable = true)
 |-- at: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- city_name: string (nullable = true)
 |-- clv_total: long (nullable = true)
 |-- country: string (nullable = true)
 |-- custom_1: string (nullable = true)
 |-- custom_2: string (nullable = true)
 |-- custom_3: string (nullable = true)
 |-- custom_4: string (nullable = true)
 |-- device_new: boolean (nullable = true)
 |-- first-accessed-page: string (nullable = true)
 |-- install_uuid: string (nullable = true)
 |-- language: string (nullable = true)
 |-- library_ver: string (nullable = true)
 |-- marketing_campaign: string (nullable = true)
 |-- marketing_medium: string (nullable = true)
 |-- mark

In [6]:
#Inspecionando os dados
dfPageViews.toPandas().head(10)

,Last Accessed Url,Page Category,Page Category 1,Page Category 2,Page Category 3,Page Name,at,browser,carrier,city_name,...,name,nth,os_ver,platform,region,session_uuid,studentId_clientType,type,user_type,uuid
0,/,perfil,perfil,Undefined,Undefined,/perfil/22482764/materiais,2017-11-16 02:10:20,Chrome 62,Telemar Norte Leste S.a.,None,...,Page View,17.0,,Linux,None,188031bec37fc43b737c2c49349076700ae89128,34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364...,e,known,1b3ed1360694ceae79f6361ed11b03cf245311c8
1,/disciplina/estudos-dirigidos-viii?type=6&mate...,busca,busca,Undefined,Undefined,/busca?disciplina=686342&disciplina=669865&dis...,2017-11-16 02:10:26,Firefox 56,Telefonica Brasil S.a.,None,...,Page View,19.0,,Windows 7,None,e3fe75ffce700fef07fd93e66312117ba008f046,23fa8b361cc7ea61dea852d72eba9129f05981077e1f65...,e,known,b0110b5beac39737f52678ffdba6c565bb72d8e2
2,/ranking/curso,disciplina,disciplina,marketing-pessoal,Undefined,/disciplina/marketing-pessoal,2017-11-16 02:10:22,Chrome 61,Tim Celular S.a.,None,...,Page View,2.0,,Windows 7,None,e0841700c23ea9aaadf1104690e7d6e12a8a771a,14cc805c72b0eec6292916da59c9c45a8aa621a1138eb6...,e,known,bea05ad3a2fc2b621bcdb048eafa4e70f5344b17
3,None,home,None,None,None,/,2017-11-16 02:10:34,Mobile Safari 10,Internexa Brasil Operadora De Telecomunicacoes...,None,...,Page View,2.0,10.3.2,iOS,None,318775acadbb0c72430ac2eeb05782589fbd1cf7,98774aeb803eb08e08020309fe8472af79033fbae26ea4...,e,known,a8ee07ca847a3126007ee0f5ceec8d6fa0d92727
4,None,disciplina,disciplina,cinesiologia,Undefined,/disciplina/cinesiologia?type=6&materialid=175...,2017-11-16 02:10:53,Chrome 61,Hipernet Servico De Comunicacao Ltda Me,None,...,Page View,11.0,,Windows 8.1,None,91bf1fbddb681569c914a8bb05fa0e260d9d92bd,122ac90aee4fc73660620456712cfea98d7bd720a1fc71...,e,known,ad6d53dc93ec8d8c30050f4391c9ecdf7da13710
5,None,disciplina,disciplina,direito-penal-i,Undefined,/disciplina/direito-penal-i?type=6&materialid=...,2017-11-16 02:10:44,Chrome 58,Telefonica Brasil S.a.,None,...,Page View,6.0,,Windows 7,None,d52b6c4881b52297521463e789d6721cb85e08b6,d799aecbb0a87cdd5412c68172189cc65d84644ac26e34...,e,known,23886a3fe7cb5853b3aeb7f3690c42ae1b31d7f3
6,None,disciplina,disciplina,fisica-i,Undefined,/disciplina/fisica-i?type=6&materialid=5191228,2017-11-16 02:10:46,Chrome 61,Digital Net Ltda,None,...,Page View,8.0,,Windows 7,None,af9ab1ed319a1a8a62dc6e39638c7ee743340eb7,ca122c896b84fa32f750f95c9c163d3e1146c63e24f2dd...,e,known,eacbfbb3d1058820a3679db024155efa9e98230b
7,None,disciplina,disciplina,introducao-ao-estudo-de-direito,Undefined,/disciplina/introducao-ao-estudo-de-direito,2017-11-16 02:10:53,Chrome 61,T-net Wireless E Informa#769tica,None,...,Page View,8.0,,Windows 10,None,74c44cbf080da372d3f63ae61bd67ed006f1a2a6,e1b0981e40fdef0c39b9e1442ca23aa10eceaee7f1013e...,e,known,7314139889cd861e86608797030931d504dda2e7
8,None,busca,None,None,None,/busca,2017-11-16 02:10:54,Mobile Safari 11,Claro Brazil,None,...,Page View,1.0,11.1.1,iOS,None,0bf547c5ac802a0e0b3e28b8d5e012066fafa9d6,859fa6b0af443a290eb97b19cc3e16eeea37cc60c05a5d...,e,known,01d36048a25a15778ccda7ffa384bd755b73ae61
9,None,home,Undefined,Undefined,Undefined,/,2017-11-16 02:11:03,Edge 14,Quality Telecom Servicos De Comunicacao Ltda Epp,None,...,Page View,2.0,,Windows 10,None,943ba5d00ab9607eb2979618e11c10ccec1a17dd,344262de5df32d62ccd1592dfd9aa073a584d0fdd1f164...,e,known,c70f886f19d653d2e000bba9e855da69acef19be


In [7]:
#Quais disciplinas os usuários mais acessaram esse mês?
dfPageViews.filter(dfPageViews['Page Category 1'] == 'disciplina') \
    .groupBy("Page Category 2") \
        .count().sort(col("count").desc())\
            .show()


+--------------------+-----+
|     Page Category 2|count|
+--------------------+-----+
|            seguindo| 3323|
|matematica-financ...| 2629|
|       direito-penal| 2245|
|       administracao| 1980|
|    ciencia-politica| 1963|
| gestao-da-qualidade| 1912|
|          bioquimica| 1830|
|metodologia-cient...| 1828|
|       direito-civil| 1765|
|           pedagogia| 1757|
|   direito-penal-iii| 1757|
|     direito-penal-i| 1711|
|fenomenos-dos-tra...| 1672|
|             direito| 1635|
|    direito-penal-ii| 1599|
|   fisiologia-humana| 1556|
|introducao-a-admi...| 1530|
|contabilidade---c...| 1515|
|direito-constituc...| 1445|
|gestao-da-cadeia-...| 1439|
+--------------------+-----+
only showing top 20 rows



In [8]:
#Por quais plataformas que os usuários acessam ?
dfPageViews.select("platform").distinct().show(truncate=False)

+--------------+
|platform      |
+--------------+
|Windows 10    |
|Windows 8     |
|iOS           |
|Windows 8.1   |
|Linux         |
|Windows 2000  |
|Other         |
|Windows RT    |
|Fedora        |
|Windows Vista |
|Chrome OS     |
|BlackBerry OS |
|Ubuntu        |
|Windows Phone |
|Android       |
|Windows XP    |
|Windows 7     |
|Windows RT 8.1|
|Mac OS X      |
+--------------+



In [9]:
#Qual plataforma mais utilizam ao acessar o PasseiDireto
dfPageViews.groupBy("platform") \
        .count().sort(col("count").desc())\
            .show()

+--------------+------+
|      platform| count|
+--------------+------+
|     Windows 7|292842|
|    Windows 10|239121|
|   Windows 8.1| 58443|
|       Android| 54825|
|           iOS| 18544|
|     Windows 8| 11164|
|      Mac OS X|  7990|
|    Windows XP|  6851|
|         Linux|  4060|
|        Ubuntu|  2099|
| Windows Phone|  1130|
| Windows Vista|   875|
|     Chrome OS|   848|
|    Windows RT|    51|
|        Fedora|    46|
|         Other|    40|
|Windows RT 8.1|     6|
|  Windows 2000|     5|
| BlackBerry OS|     2|
+--------------+------+



In [10]:
#Para cruzar os dados com Students.json, é necessário retirar o final : @Website ou @Webapp. Antes de fazer essa etapa, eu havia realizado o join e não houve efeito.O motivo está nessa linha.
dfPageViews.select('studentId_clientType').distinct().limit(10).collect()

[Row(studentId_clientType='128ce8b5dbec4fabac5b5b1d457ea36f0cddd2ea46ae1a7e5921ff97714030f0@Website'),
 Row(studentId_clientType='bd9da6395c2ed8fa60f98afca69b838c19ec13436e504fe644b2442d1d4c879b@Website'),
 Row(studentId_clientType='d094824fca5c0022eca647a0e6511aebda1aef684d9dc3e8952f91a2df283b8f@Website'),
 Row(studentId_clientType='ac6f809c2f3ad6b535808d81beaddf8fb873db75c8bde3b1478e7bab5f53775f@Website'),
 Row(studentId_clientType='bdfed2849b252d44ea12352dc8b82874148ca8175bd91d0ce18ab2e48a39264e@Website'),
 Row(studentId_clientType='6e4028ec6bda9b6072ffac3d988d502473d8788e6b640fe85e766a6eabb8b273@Website'),
 Row(studentId_clientType='e09a9098a5e26f3ad53eb32da36894aa3e462109cca36fa8790efdf59b26e37e@Website'),
 Row(studentId_clientType='e9ffbf05f681c55fa5986ed2451565c5c077678d4edc7cd6e3c71e43fa25a740@Website'),
 Row(studentId_clientType='c302f85e299a7a39774a2caf652a4bb8ea3355e37a53b0d31a0817387a3cb37a@Website'),
 Row(studentId_clientType='421ce66f8d68668db0b760b2a4b2cc8ca2a74658d65892

In [11]:
#A A função irá criar 2 colunas novas ao dataFrame
dfPageViews=dfPageViews.withColumn("studentId",split(col("studentId_clientType"),"@").getItem(0)) \
        .withColumn("clientType",split(col("studentId_clientType"),"@").getItem(1))

dfPageViews.select('studentId').limit(1).collect()

[Row(studentId='34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364e097371db586d4ea48')]

In [12]:
#Por qual Client os usuários mais navegam ? Podemos analisar aqui por onde o consumo e navegação é maior.
dfPageViews.groupBy("clientType") \
        .count().sort(col("count").desc())\
            .show()

+----------+------+
|clientType| count|
+----------+------+
|   Website|577073|
|      null| 65750|
|    Webapp| 56119|
+----------+------+



In [13]:
subscriptions = spark.read.json(['Datasets - Teste Data Engineer - Passei Direto/BASE A/subscriptions.json'])

In [14]:
left_join =  dfPageViews.join(subscriptions, dfPageViews.studentId == subscriptions.StudentId,how='left' )


In [15]:
left_join.limit(1).collect()

[Row(Last Accessed Url='/', Page Category='perfil', Page Category 1='perfil', Page Category 2='Undefined', Page Category 3='Undefined', Page Name='/perfil/22482764/materiais', at='2017-11-16 02:10:20', browser='Chrome 62', carrier='Telemar Norte Leste S.a.', city_name=None, clv_total=None, country='br', custom_1='ESTÁCIO EAD', custom_2='Pedagogia', custom_3=None, custom_4='Core User', device_new=False, first-accessed-page=None, install_uuid='fdfff303505f8a18b17ee40587e785f6bb9c8374', language='pt', library_ver='web_3.3.3', marketing_campaign=None, marketing_medium=None, marketing_source=None, model='Linux armv7l', name='Page View', nth=17, os_ver='', platform='Linux', region=None, session_uuid='188031bec37fc43b737c2c49349076700ae89128', studentId_clientType='34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364e097371db586d4ea48@Website', type='e', user_type='known', uuid='1b3ed1360694ceae79f6361ed11b03cf245311c8', studentId='34cbeaf4a28c798de94cd9afb43d4e2e49ce80d6b52364e097371db586d4ea48', 

In [16]:
#Consigo Entender o comportamento dos usuários de acordo com o plano.
left_join.groupBy("PlanType") \
        .count().sort(col("count").desc())\
            .show()


+--------+------+
|PlanType| count|
+--------+------+
|    null|696407|
|  Mensal|  2314|
|   Anual|   222|
+--------+------+



In [25]:
#Consigo identificar o que meus usuários de planos mensais ou anuais geralmente acessam , o que mais acessaram esse mês.
left_join.filter(left_join['PlanType'] == 'Mensal') \
    .groupBy("custom_2") \
        .count().sort(col("count").desc())\
            .show()

+--------------------+-----+
|            custom_2|count|
+--------------------+-----+
|             Direito|  392|
|           Pedagogia|  333|
|       Administração|  269|
|            Nutrição|  112|
|Engenharia de Pro...|  110|
|Contabilidade / C...|  105|
|                null|   84|
|Gestão de Comérci...|   81|
| Engenharia Elétrica|   69|
|   Gestão Financeira|   60|
|           Logística|   58|
|Enfermagem e Obst...|   57|
|Gestão da Informação|   48|
|Análise e Desenvo...|   45|
|Gestão de Process...|   38|
|Sistemas de Infor...|   37|
|Letras - Portuguê...|   30|
|    Engenharia Civil|   28|
|Gestão da Tecnolo...|   28|
|         Odontologia|   27|
+--------------------+-----+
only showing top 20 rows

